In [63]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import os
import torch

from dataclasses import asdict

from flow_mnist.data import MNISTDataset
from flow_mnist.config import load_config

In [65]:
torch.manual_seed(42)
os.chdir("..") if not os.path.exists("config.yaml") else None
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [66]:
conf = load_config('config.yaml')
dataset = MNISTDataset(**asdict(conf.data))